# Avazu

**What this notebook does**
Interactive, reproducible EDA for the Avazu CTR dataset using **Plotly** with best-practice checks and corrections.  
It extends the prior version with: robust categorical casting, train–test schema/drift preview, clustered correlations,
date rollups, ANOVA/Kruskal tests, missingness↔target diagnostics with CIs, monotonicity hints, and an imputation strategy inventory.

---

### Table of Contents
1. [Config & Imports](#config)  
2. [Changelog & Corrections](#changelog)  
3. [Utilities (logging, memory, stats)](#utils)  
4. [Load Data (typed, memory-optimized)](#load)  
5. [Data Overview](#overview)  
6. [Missingness Analysis](#missingness)  
7. [Target-Aware Summary](#targetaware)  
8. [Univariate Analysis](#univariate)  
9. [Bivariate & Multivariate Analysis](#multivariate)  
10. [Time Series Handling](#timeseries)  
11. [Train–Test Schema & Drift Preview](#drift)  
12. [Date Feature Rollups](#daterollups)  
13. [Mixed Tests: ANOVA & Kruskal](#anova)  
14. [Missingness ↔ Target Diagnostics](#missxtgt)  
15. [Clustered Correlation Heatmap](#clusteredcorr)  
16. [Monotonicity Hints vs Time](#monotonicity)  
17. [Imputation Strategy Inventory (Not Recommendations)](#imputeinventory)  
18. [Feature Quality Checks](#quality)  
19. [Appendix (helpers)](#appendix)

---

### How to interpret this notebook overall
- Confirm schema and memory profile; watch for high-cardinality identifiers and degenerate columns.  
- Review missingness and co-missingness; examine any association with the target (diagnostic only).  
- Inspect univariate distributions; for categorical, look at top-k and rare levels.  
- Review bivariate associations (numeric/categorical) and correlation structure (clustered).  
- For time-based features, verify continuity, gaps, and periodic rollups.  
- No subjective interpretation or modeling guidance is provided.


In [ ]:
import time
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from scipy.cluster.hierarchy import leaves_list, linkage
from scipy.stats import chi2_contingency, f_oneway, kruskal, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

In [ ]:
# ---------- Paths ----------
DATA_DIR = Path("../data/raw/avazu")
TRAIN_FILE = DATA_DIR / "train.gz"
TEST_FILE = DATA_DIR / "test.gz"

# ---------- Core Columns (Avazu) ----------
ID_COL = "id"
TARGET_COL = "click"
DATETIME_COL = "hour"
DATETIME_FORMAT = "%y%m%d%H"

# ---------- Reading & Sampling ----------
SAMPLE_FRAC = 0.05
RANDOM_STATE = 42
LOW_MEMORY = True

# ---------- Casting Controls ----------
DTYPE_COLS = {
    "id": np.int64,
    "click": np.int8,
    "hour": np.int64,
    "C1": np.int32,
    "banner_pos": np.int8,
    "site_id": "category",
    "site_domain": "category",
    "site_category": "category",
    "app_id": "category",
    "app_domain": "category",
    "app_category": "category",
    "device_id": "category",
    "device_ip": "category",
    "device_model": "category",
    "device_type": np.int8,
    "device_conn_type": np.int8,
    "C14": np.int32,
    "C15": np.int32,
    "C16": np.int32,
    "C17": np.int32,
    "C18": np.int32,
    "C19": np.int32,
    "C20": np.int32,
    "C21": np.int32,
}
ALL_COLS = list(DTYPE_COLS.keys())

# ---------- Plotting & Reporting ----------
MAX_UNIVAR_PLOTS = 12
TOP_K_CATEGORIES = 20
RARE_LEVEL_THRESHOLD = 0.01
HEAVY_PLOT_SAMPLE = 500_000
CARDINALITY_BUCKET_LABELS = {
    0: "very_low",
    1: "low",
    2: "medium",
    3: "high",
    4: "very_high",
}

# ---------- Correlations & Stats ----------
CORR_METHODS = ("pearson", "spearman")
HIGH_CORR_THRESHOLD = 0.9
CRAMERS_V_MAX_CAT_LEVELS = 30
THEIL_U_MAX_CAT_LEVELS = 30
OTHER_LABEL = "__OTHER__"

# ---------- Time Series ----------
DECOMPOSE_PERIOD = 24
ROLLING_WINDOW = 24

# ---------- Performance ----------
DISPLAY_SAMPLES = 10_000
VIF_MAX_NUMERIC = 20

In [ ]:
def log(msg: str):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}")

#### Load Data

---

**Interpretation:** Confirm expected schema, dtypes, and memory usage. Casting is conditional for potentially high-cardinality IDs.


In [ ]:
def read_csv_infer(path, usecols=None):
    return pd.read_csv(
        path,
        usecols=usecols,
        dtype=DTYPE_COLS,
        low_memory=LOW_MEMORY,
        compression="infer",
    )


def memory_usage_mb(df: pd.DataFrame) -> float:
    return df.memory_usage(deep=True).sum() / (1024**2)


def downcast_numeric(df: pd.DataFrame) -> pd.DataFrame:
    for c in df.columns:
        if pd.api.types.is_integer_dtype(df[c]):
            df[c] = pd.to_numeric(df[c], downcast="integer")
        elif pd.api.types.is_float_dtype(df[c]):
            df[c] = pd.to_numeric(df[c], downcast="float")

    return df


def parse_hour_to_datetime(series: pd.Series) -> pd.Series:
    ser = series.astype(str).str.zfill(8)
    return pd.to_datetime(ser, format=DATETIME_FORMAT, errors="coerce", utc=True)


log("Reading Avazu data...")

df = read_csv_infer(TRAIN_FILE, usecols=ALL_COLS)
log(f"Train read: shape={df.shape}, mem={memory_usage_mb(df):.2f} MB")

df_test = read_csv_infer(TEST_FILE, usecols=[c for c in ALL_COLS if c != TARGET_COL])
log(f"Test read: shape={df_test.shape}, mem={memory_usage_mb(df_test):.2f} MB")

df[DATETIME_COL] = parse_hour_to_datetime(df[DATETIME_COL])

df = downcast_numeric(df)
log(f"After downcast: mem={memory_usage_mb(df):.2f} MB")

df = df.sample(frac=SAMPLE_FRAC, random_state=RANDOM_STATE).reset_index(drop=True)
log(
    f"Sampled df to fraction={SAMPLE_FRAC}: shape={df.shape}, mem={memory_usage_mb(df):.2f} MB"
)

#### Data Overview

---

**Interpretation:** Verify row/column counts, dtypes, memory footprint; identify duplicates, constant & near-zero-variance columns.

In [ ]:
def near_zero_variance(df: pd.DataFrame, threshold: float = 0.95) -> pd.DataFrame:
    rows = []

    for c in df.columns:
        vc = df[c].value_counts(dropna=False)
        top_share = (vc.iloc[0] / len(df)) if len(vc) else np.nan
        rows.append(
            {
                "column": c,
                "top_value_share": float(top_share),
                "near_zero_var": bool(top_share >= threshold),
            }
        )

    return pd.DataFrame(rows).sort_values("top_value_share", ascending=False)


log("Overview...")

print("Shape:", df.shape)
print("Dtypes:\n", df.dtypes)

print("Head:")
display(df.head(5))
print("Tail:")
display(df.tail(5))

const_cols = [c for c in df.columns if df[c].nunique(dropna=False) <= 1]
print("Constant columns:", const_cols)

nzv = near_zero_variance(df)
display(nzv.sort_values("near_zero_var", ascending=False).head(20))

n_dup = df.duplicated().sum()
print(f"Duplicate rows: {n_dup}")

#### Missingness Analysis

---

**Interpretation:** Inspect high-missing columns, co-missingness patterns, and row-level missingness. Patterns may hint MNAR (not assessed).

In [ ]:
log("Missingness analysis...")

miss_pct = df.isna().mean().sort_values(ascending=False) * 100.0
miss_tbl = pd.DataFrame({"column": miss_pct.index, "missing_%": miss_pct.values})
display(miss_tbl)

#### Nullish Analysis

---

**Interpretation**: ....

In [ ]:
print("Nullish analysis...")

null_pct = df.isnull().mean().sort_values(ascending=False) * 100.0
null_tbl = pd.DataFrame({"column": null_pct.index, "nullish_%": null_pct.values})
display(null_tbl)

#### Target-Aware Summary (Classification)

---

**Interpretation:** Inspect class balance, trivial leakage checks (perfect predictors), and preview splits (random/time-based) without modeling.


In [ ]:
def is_perfect_predictor(x: pd.Series, y: pd.Series) -> bool:
    tmp = pd.DataFrame({"x": x, "y": y})
    g = tmp.groupby("x", observed=False)["y"]

    if (g.nunique() > 1).any():
        return False

    return True


log("Target-aware summary...")

tgt = df[TARGET_COL]
print("Target dtype:", tgt.dtype)
print("Target value counts:")
display(
    tgt.value_counts(dropna=False)
    .to_frame("count")
    .assign(share=lambda x: x["count"] / len(tgt))
)

px.histogram(tgt, title=f"Target Distribution — {TARGET_COL}", nbins=2).show()

perfect_cols = []
for c in df.columns:
    if c == TARGET_COL:
        continue

    if is_perfect_predictor(df[c], tgt):
        perfect_cols.append(c)

print("Columns that perfectly predict target (potential leakage):", perfect_cols)

#### Univariate Analysis

---

**Interpretation:** Numeric: skew & outlier %, hist/box/violin. Categorical: frequency, top-k, rare levels, high-card flags. Text: lengths & tokens.

In [ ]:
log("Univariate analysis...")

num_cols = (
    df.select_dtypes(include=[np.number]).drop(columns=[TARGET_COL]).columns.tolist()
)
cat_cols = df.select_dtypes(include=["category"]).columns.tolist()

print("Numeric columns (excl. target):", len(num_cols))
print("Categorical-like columns:", len(cat_cols))

In [ ]:
def robust_outlier_stats(s: pd.Series) -> dict:
    # Skewness
    sk = skew(s, bias=False)

    # IQR outlier
    q1, q3 = np.percentile(s, [25, 75])
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    iqr_outlier = ((s < lower) | (s > upper)).mean() * 100.0

    # Z-Score outlier
    z = np.abs(stats.zscore(s, nan_policy="omit"))
    z_outlier = (z > 3).mean() * 100.0

    return {
        "skewness": float(sk),
        "iqr_outlier_pct": float(iqr_outlier),
        "zscore_outlier_pct": float(z_outlier),
    }


def classify_cardinality_bucket(
    unique_cnt: int,
    unique_rt: float,
    a_thresholds: tuple[int],
    r_thresholds: tuple[float],
) -> str:
    def absolute_level(count: int) -> int:
        a0, a1, a2, a3 = a_thresholds

        if count <= a0:
            return 0
        if count <= a1:
            return 1
        if count <= a2:
            return 2
        if count <= a3:
            return 3

        return 4

    def relative_level(ratio: float) -> int:
        r0, r1, r2, r3 = r_thresholds

        if ratio < r0:
            return 0
        if ratio < r1:
            return 1
        if ratio < r2:
            return 2
        if ratio < r3:
            return 3

        return 4

    level = max(absolute_level(unique_cnt), relative_level(unique_rt))
    return CARDINALITY_BUCKET_LABELS[level]


# --- Numeric summaries + plots ---
num_report = []
for c in num_cols:
    s = pd.to_numeric(df[c], errors="coerce").astype(float)
    out = robust_outlier_stats(s)

    num_report.append({"column": c, **out})

# --- Categorical summaries ---
cat_report = []
for c in cat_cols:
    vc = df[c].value_counts()

    topk = vc.head(TOP_K_CATEGORIES).to_frame("count")
    topk["share"] = topk["count"] / len(df)

    unique_cnt = int(df[c].nunique())
    unique_rt = unique_cnt / len(df)

    cardinality_bucket = classify_cardinality_bucket(
        unique_cnt=unique_cnt,
        unique_rt=unique_rt,
        a_thresholds=(10, 100, 1_000, 100_000),
        r_thresholds=(0.0001, 0.001, 0.01, 0.1),
    )

    rare_mask = vc / len(df) < RARE_LEVEL_THRESHOLD
    rare_levels_share = (vc[rare_mask].sum() / len(df)) if rare_mask.any() else 0.0

    px.bar(
        topk.reset_index().rename(columns={"index": c}),
        x=c,
        y="count",
        title=f"Top-{TOP_K_CATEGORIES} — {c}",
    ).show()
    cat_report.append(
        {
            "column": c,
            "unique": unique_cnt,
            "cardinality_bucket": cardinality_bucket,
            "rare_levels_share": float(rare_levels_share),
        }
    )

display(pd.DataFrame(num_report))
display(pd.DataFrame(cat_report))

#### Bivariate & Multivariate Analysis

---

**Interpretation:** Numeric↔target (box/violin + decile lift with CIs), cat↔target (contingency with χ²/Fisher), num↔num correlations, cat↔cat (Cramér’s V, Theil’s U).


In [ ]:
log("Bivariate & multivariate...")

In [ ]:
def wilson_ci(k: float, n: int, z: float) -> tuple[float, float]:
    denominator = 1 + z**2 / n
    center_adj = k + z**2 / (2 * n)
    term = z * ((k * (1 - k)) / n + z**2 / (4 * n**2)) ** 0.5

    ci_low = ((center_adj - term) / denominator).fillna(0)
    ci_high = ((center_adj + term) / denominator).fillna(1)
    return ci_low, ci_high


# Numeric ↔ Target
base_rate = df[TARGET_COL].mean()
for c in num_cols:
    bin_labels = pd.qcut(df[c], q=10, duplicates="drop").astype(str)
    bins = pd.qcut(df[c], q=10, labels=False, duplicates="drop")

    grp = (
        df.groupby(bins)[TARGET_COL]
        .agg(["sum", "count"])
        .rename(columns={"sum": "clicks", "count": "n"})
    )
    grp["ctr"] = grp["clicks"] / grp["n"]
    grp["lift"] = grp["ctr"] / base_rate

    z = stats.norm.ppf(1 - 0.05 / 2)
    ci_low, ci_high = wilson_ci(grp["ctr"], grp["n"], z)
    grp["ci_low"] = ci_low
    grp["ci_high"] = ci_high
    display(grp)

    x_axis = list(range(len(grp)))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_axis, y=grp["ctr"], mode="lines+markers", name="CTR"))
    fig.add_trace(
        go.Scatter(
            x=x_axis + x_axis[::-1],
            y=list(grp["ci_high"]) + list(grp["ci_low"])[::-1],
            fill="toself",
            fillcolor="rgba(0,100,80,0.2)",
            line=dict(color="rgba(255,255,255,0)"),
            hoverinfo="skip",
            showlegend=False,
            name="95% CI",
        )
    )
    fig.update_layout(
        title=f"CTR by {c} Quantiles (with 95% Wilson CI)",
        xaxis_title=f"{c} Quantile Bin (ordered low to high)",
        yaxis_title="Click-Through Rate (CTR)",
        xaxis=dict(tickvals=x_axis, ticktext=grp.index),
    )
    fig.show()

In [ ]:
# Categorical ↔ Target
base_rate = float(df[TARGET_COL].mean())
for c in cat_cols:
    s = df[c]

    vc = s.value_counts()
    keep_idx = vc.nlargest(TOP_K_CATEGORIES).index
    keep_idx = vc[vc >= 100_000].index.union(keep_idx)
    if len(keep_idx) == 0 and len(vc) > 0:
        keep_idx = vc.nlargest(min(5, len(vc))).index

    if OTHER_LABEL not in s.cat.categories:
        s = s.cat.add_categories([OTHER_LABEL])

    collapsed = s.where(s.isin(keep_idx), OTHER_LABEL)

    tmp = pd.DataFrame({c: collapsed, TARGET_COL: df[TARGET_COL]})
    counts = pd.crosstab(tmp[c], tmp[TARGET_COL])

    col_idx = pd.Index(sorted(tmp[TARGET_COL].unique()), name=TARGET_COL)
    counts = counts.reindex(columns=col_idx, fill_value=0)

    chi2, p, dof, exp = chi2_contingency(counts.values)
    exp_min = float(exp.min())
    log(f"[{c}] χ²={chi2:.3f}, dof={dof}, p={p:.3g}, min_expected={exp_min:.2f}")

    rate_tbl = (
        tmp.groupby(c, observed=True)[TARGET_COL]
        .agg(ctr="mean", count="size")
        .assign(lift=lambda t: t["ctr"] / base_rate if base_rate > 0 else float("nan"))
        .sort_values(["count", "ctr"], ascending=[False, False])
    )
    display(rate_tbl)

    plot_df = rate_tbl.nlargest(TOP_K_CATEGORIES, "count").sort_values(
        "ctr", ascending=False
    )
    fig = px.bar(
        plot_df.reset_index(),
        x=c,
        y="ctr",
        title=f"Target rate by {c} (top {len(plot_df)}; tail→{OTHER_LABEL})",
    )
    fig.update_layout(xaxis={"categoryorder": "total descending", "tickangle": -45})
    fig.show()

In [ ]:
# Numeric ↔ Numeric
for method in CORR_METHODS:
    num_df = df[num_cols]
    corr = num_df.corr(method=method)

    px.imshow(
        corr, aspect="auto", text_auto=".2f", title=f"{method.title()} Correlations"
    ).show()

    pairs = []
    cols = corr.columns
    for i in range(len(cols)):
        for j in range(i + 1, len(cols)):
            v = corr.iloc[i, j]

            if abs(v) >= HIGH_CORR_THRESHOLD:
                pairs.append((cols[i], cols[j], float(v)))

    if pairs:
        display(
            pd.DataFrame(pairs, columns=["col1", "col2", "corr"]).sort_values(
                "corr", ascending=False
            )
        )

In [ ]:
def cramers_v_corrected(confusion: pd.DataFrame) -> float:
    chi2 = chi2_contingency(confusion)[0]
    n = confusion.values.sum()

    phi2 = chi2 / n
    r, k = confusion.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    denom = min((kcorr - 1), (rcorr - 1))

    return float(np.sqrt(phi2corr / denom))


def theils_u(x: pd.Series, y: pd.Series) -> float:
    def entropy(s: pd.Series) -> float:
        p = s.value_counts(normalize=True, dropna=False)
        return float(stats.entropy(p, base=2))

    s_xy = pd.crosstab(x, y)
    pxy = s_xy / s_xy.values.sum()
    py = pxy.sum(axis=0)

    h_x = entropy(x)
    h_x_given_y = 0.0

    for yv in py.index:
        p_y = py.loc[yv]

        if p_y > 0:
            p_x_given_y = pxy.loc[:, yv] / p_y
            h_x_given_y += p_y * stats.entropy(p_x_given_y.fillna(0), base=2)

    return float((h_x - h_x_given_y) / h_x)


# Categorical ↔ Categorical
small_cats = [c for c in cat_cols if df[c].nunique() <= 100_000]
pairs, pairs_u = [], []

for i in range(len(small_cats)):
    for j in range(i + 1, len(small_cats)):
        a, b = small_cats[i], small_cats[j]
        ct = pd.crosstab(df[a], df[b])

        if ct.shape[0] >= 2 and ct.shape[1] >= 2:
            v = cramers_v_corrected(ct)
            pairs.append((a, b, float(v)))

        if (
            df[a].nunique() <= 50_000
            and df[b].nunique() <= 50_000
        ):
            # Symmetrize by averaging U(X|Y) and U(Y|X)
            uab = theils_u(df[a], df[b])
            uba = theils_u(df[b], df[a])
            pairs_u.append((a, b, float((uab + uba) / 2)))

if pairs:
    display(
        pd.DataFrame(pairs, columns=["cat1", "cat2", "cramers_v"]).sort_values(
            "cramers_v", ascending=False
        )
    )

if pairs_u:
    display(
        pd.DataFrame(pairs_u, columns=["cat1", "cat2", "theils_u_avg"]).sort_values(
            "theils_u_avg", ascending=False
        )
    )

#### Time Series Handling

---

**Interpretation:** Inspect continuity, decomposition, stationarity, rolling stats. No forecasting.


In [ ]:
log("Time series diagnostics on aggregated CTR by hour or counts...")

ts = (
    df.set_index(DATETIME_COL)
    .groupby(pd.Grouper(freq="h"))[TARGET_COL]
    .mean()
    .rename("rate")
)
y_label = "CTR"

display(ts.describe())

full_idx = pd.date_range(ts.index.min(), ts.index.max(), freq="h", tz="UTC")
missing_periods = full_idx.difference(ts.index)
print("Missing hourly periods:", len(missing_periods))
print("Timezone:", ts.index.tz)

px.line(
    ts.reset_index(), x=DATETIME_COL, y=ts.name, title="Observed time series"
).update_layout(yaxis_title=y_label).show()

res = seasonal_decompose(
    ts,
    period=DECOMPOSE_PERIOD,
    model="additive",
    two_sided=False,
    extrapolate_trend="freq",
)
for y, part_name, part_series in [
    ("trend", "Trend", res.trend),
    ("seasonal", "Seasonality", res.seasonal),
    ("resid", "Residual", res.resid),
]:
    px.line(part_series.reset_index(), x=DATETIME_COL, y=y, title=part_name).show()

series = ts.values
result = adfuller(series, autolag="AIC")
print(
    "ADF:",
    {"stat": result[0], "pvalue": result[1], "lags": result[2], "nobs": result[3]},
)

rmean = ts.rolling(ROLLING_WINDOW).mean().rename("rolling_mean")
rstd = ts.rolling(ROLLING_WINDOW).std().rename("rolling_std")
tmp = pd.concat([ts, rmean, rstd], axis=1).reset_index()
px.line(
    tmp,
    x=DATETIME_COL,
    y=[ts.name, "rolling_mean", "rolling_std"],
    title=f"Rolling mean/std (window={ROLLING_WINDOW})",
).show()

#### Date Feature Rollups

---

**Interpretation:** Roll-ups by hour-of-day, day-of-week; DOW×HOD heatmap (CTR or counts).

In [ ]:
tmp = df.copy()
tmp["hod"] = tmp[DATETIME_COL].dt.hour
tmp["dow"] = tmp[DATETIME_COL].dt.dayofweek

hod = tmp.groupby("hod")[TARGET_COL].mean().reset_index()
dow = tmp.groupby("dow")[TARGET_COL].mean().reset_index()
mat = tmp.groupby(["dow", "hod"])[TARGET_COL].mean().unstack(0)
y_label = "CTR"

px.bar(hod, x="hod", y=hod.columns[1], title=f"Hour-of-day {y_label}").show()
px.bar(dow, x="dow", y=dow.columns[1], title=f"Day-of-week {y_label}").show()
px.imshow(
    mat.transpose(),
    aspect="auto",
    text_auto=".2f",
    title=f"DOW x HOD heatmap ({y_label})",
).show()

#### Train–Test Schema & Drift Preview

---

**Interpretation:** Compare basic schema, dtype consistency, and category overlaps; report unseen categories (test vs train).


In [ ]:
log("Train-Test schema & drift preview...")

common_cols = [c for c in df.columns if c in df_test.columns]
dtypes_cmp = pd.DataFrame(
    {
        "train_dtype": df[common_cols].dtypes.astype(str),
        "test_dtype": df_test[common_cols].dtypes.astype(str),
    }
)
dtypes_cmp["match"] = dtypes_cmp["train_dtype"] == dtypes_cmp["test_dtype"]
display(dtypes_cmp)

drift_rows = []
for c in common_cols:
    if str(df[c].dtype) in ("category", "object", "string"):
        tr_levels = set(df[c].unique())
        te_levels = set(df_test[c].unique())

        unseen_in_test = len(te_levels - tr_levels)
        unseen_rate = unseen_in_test / max(len(te_levels), 1)
        drift_rows.append(
            {
                "column": c,
                "train_levels": len(tr_levels),
                "test_levels": len(te_levels),
                "unseen_in_test": unseen_in_test,
                "unseen_rate": float(unseen_rate),
            }
        )

if drift_rows:
    display(
        pd.DataFrame(drift_rows).sort_values("unseen_rate", ascending=False).head(30)
    )

#### Mixed Tests: ANOVA & Kruskal (stats only)

---

**Interpretation:** For selected numeric vs categorical pairs (top-k categories), report test statistics only. No interpretation/conclusions.


In [ ]:
for num in num_cols:
    for cat in cat_cols:
        vc = df[cat].value_counts().head(TOP_K_CATEGORIES).index
        gg = [df.loc[df[cat]==lvl, num] for lvl in vc]
        gg = [g for g in gg if len(g) > 1]

        if len(gg) < 2:
            continue

        fstat, fp = f_oneway(*gg)
        kstat, kp = kruskal(*gg)
        log(f"{num} ~ {cat} | ANOVA F={fstat:.3g}, p={fp:.3g} | Kruskal H={kstat:.3g}, p={kp:.3g}")

#### Clustered Correlation Heatmap

---

**Interpretation:** Hierarchically cluster the correlation matrix to reveal blocks of related features.


In [ ]:
num_df = df[num_cols].drop(columns=[ID_COL])

corr = num_df.corr(method="pearson").fillna(0)
dist = 1 - corr.abs()
Z = linkage(dist, method="average")
order = leaves_list(Z)
corr_ord = corr.iloc[order, :].iloc[:, order]

fig = px.imshow(
    corr_ord, aspect="auto", text_auto=".2f", title="Correlation Heatmap"
).show()

#### Monotonicity Hints vs Time

---

**Interpretation:** Spearman correlation between features and chronological order (monotonic trends).


In [ ]:
order = df[DATETIME_COL].rank(method="first").astype(float)
rows = []

for c in num_cols + cat_cols:
    s = df[c]
    rho, p = stats.spearmanr(order.loc[s.index], s, nan_policy="omit")
    rows.append(
        {
            "column": c,
            "spearman_rho_vs_time": float(rho),
            "p_value": float(p),
        }
    )

if rows:
    display(pd.DataFrame(rows).sort_values("spearman_rho_vs_time", ascending=False))

#### Feature Quality Checks

---

**Interpretation:** Recap missingness, cardinality, multicollinearity (VIF), constant columns, and potential leakage flags.

In [ ]:
def summarize_cardinality(df: pd.DataFrame) -> pd.DataFrame:
    n = len(df)
    rows = []

    for c in df.columns:
        u = df[c].nunique(dropna=True)
        rows.append(
            {
                "column": c,
                "dtype": str(df[c].dtype),
                "unique_count": int(u),
                "unique_ratio": float(u / max(n, 1)),
            }
        )

    return pd.DataFrame(rows).sort_values("unique_count", ascending=False)


log("Feature quality checks...")

card = summarize_cardinality(df)
display(card.head(20))

num_cols_for_vif = df[num_cols].columns.to_list()

X = df[num_cols_for_vif].astype(float)
X = X.loc[:, X.std() > 0]
X = (X - X.mean()) / X.std()

vifs = []
for i, c in enumerate(X.columns):
    v = variance_inflation_factor(X.values, i)
    vifs.append(
        {
            "column": c,
            "VIF": float(v),
        }
    )

display(pd.DataFrame(vifs).sort_values("VIF", ascending=False))